# Overview

This is the pipeline in charge of the bge-base-en-1.5v mode fine-tune. In this notebook what I'll do is the following:
1. download train and validation data from the previous step from S3
2. fine-tune the model for a couple of rounds
3. compare the fine-tuned model with other versions of the same model (to do so I create a faiss index for each and run a serach of 10 chunks per query to then calculate several metrics)
4. save the embeddings generated by the fine-tuned embedding model to OpenSearch

---

# Setup

In [1]:
!pip install -q -r model_training/requirements.txt

In [2]:
from general_utils import load_config
import torch

CONFIG = load_config()
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from general_utils import S3Manager
s3_client = S3Manager.get_client()

In [4]:
S3Manager.download_files(
    s3_client,
    'model_training/data',
     ["training.json",
     "test_queries.jsonl",
     "corpus.jsonl",
     "test_qrels.jsonl"
    ],
    "medical-qa-data")

----

# Fine tuning
Will utilize the FlagEmbedding library implementation of the tuning pipeline, as it it suggested by the model's developers.

In [5]:
import subprocess

lr = float(CONFIG['LR'])
epochs = int(CONFIG['EPOCHS'])
warmup = float(CONFIG['WARMUP_RATIO'])
batch_size = int(CONFIG['BATCH_SIZE'])
embedding_model = CONFIG['EMBEDDING_MODEL']
query_instruction = CONFIG["QUERY_INSTRUCTION_AT_RETRIEVAL"]

cmd = [
    "torchrun", "--nproc_per_node", "1",
    "-m", "FlagEmbedding.finetune.embedder.encoder_only.base",
    "--model_name_or_path", f"BAAI/{embedding_model}",
    "--train_data", "model_training/data/training.json",
    "--query_instruction_for_retrieval", query_instruction,
    "--output_dir", "model_training/model",
    "--learning_rate", str(lr),
    "--fp16",
    "--num_train_epochs", str(epochs),
    "--per_device_train_batch_size", str(batch_size),
    "--query_max_len", "256",
    "--passage_max_len", "512",
    "--warmup_ratio", str(warmup),
    "--normalize_embeddings", "True",
    "--logging_steps", "10",
]

print(" ".join(cmd))
result = subprocess.run(cmd, check=True)

torchrun --nproc_per_node 1 -m FlagEmbedding.finetune.embedder.encoder_only.base --model_name_or_path BAAI/bge-base-en-v1.5 --train_data model_training/data/training.json --query_instruction_for_retrieval Represent this sentence for searching relevant passages: --output_dir model_training/model --learning_rate 1e-05 --fp16 --num_train_epochs 1 --per_device_train_batch_size 8 --query_max_len 256 --passage_max_len 512 --warmup_ratio 0.05 --normalize_embeddings True --logging_steps 10


2025-07-11 23:27:17.955527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752276437.971267    2309 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752276437.976325    2309 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-11 23:27:17.991838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
07/11/2025 23:27:21 - WARNING - FlagEmbedding.abc.finetune.embedder.AbsRunner -   Process rank: 0, device: 

{'loss': 0.2228, 'grad_norm': 0.2872343063354492, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.01}


  1%|▏         | 20/1498 [00:13<16:05,  1.53it/s]

{'loss': 0.4406, 'grad_norm': 0.5305854082107544, 'learning_rate': 2.133333333333334e-06, 'epoch': 0.01}


...

 99%|█████████▉| 1490/1498 [16:36<00:05,  1.53it/s]

{'loss': 0.0028, 'grad_norm': 0.0001386524672852829, 'learning_rate': 8.432888264230499e-08, 'epoch': 0.99}


100%|██████████| 1498/1498 [16:41<00:00,  1.81it/s]07/11/2025 23:44:07 - INFO - FlagEmbedding.finetune.embedder.encoder_only.base.trainer -   Saving model checkpoint to model_training/model/checkpoint-1498
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
100%|██████████| 1498/1498 [16:53<00:00,  1.81it/s]

{'train_runtime': 1013.7359, 'train_samples_per_second': 11.818, 'train_steps_per_second': 1.478, 'train_loss': 0.039941850994951095, 'epoch': 1.0}


100%|██████████| 1498/1498 [16:54<00:00,  1.48it/s]
07/11/2025 23:44:20 - INFO - FlagEmbedding.finetune.embedder.encoder_only.base.trainer -   Saving model checkpoint to model_training/model
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[rank0]:[W711 23:44:25.063887746 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


Save the model files to S3 for later usage

In [6]:
import os
file_list = []
for root, dirs, files in os.walk('model_training/model/'):
    if root=='model_training/model/':
        for file in files:
            file_list.append(file)

S3Manager.upload_bulk(
    s3_client,
    'model_training/model/',
    file_list,
    "medical-qa-data",
    "finetuned_model/"
)

---

# Evaluate
First, I'll load and generate the needed files

In [7]:
from datasets import load_dataset

queries = load_dataset("json", data_files="model_training/data/test_queries.jsonl")["train"]
corpus = load_dataset("json", data_files="model_training/data/corpus.jsonl")["train"]
qrels = load_dataset("json", data_files="model_training/data/test_qrels.jsonl")["train"]

queries_text = queries["text"]
corpus_text = [text for text in corpus["text"]]
qrels_dict = {}
for line in qrels:
    if line['qid'] not in qrels_dict:
        qrels_dict[str(line['qid'])] = {}
    for doc in line['docid']:
        qrels_dict[str(line['qid'])][str(doc)] = line['relevance']

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

I will now load the needed metrics from FlagEmbedding, the wrapper to load models with the library and a custom made wrapper "Validator" in charge of processing the needed steps to embed and index (in faiss) the queries and corpus, to then perform the semantic search to bring the top 10 neighbors.

In [10]:
!pip install -q opensearch_py==3.0.0

As we will use this retrieval without any reranking, I will focus my analysis to 1 and 3 k (nevertheless I compute 5 & 10 too)

In [15]:
from FlagEmbedding.abc.evaluation.utils import evaluate_metrics, evaluate_mrr
from FlagEmbedding import FlagModel
from model_training.utils import Validator

k_values = [1, 3, 5, 10]
raw_name = f"BAAI/{CONFIG['EMBEDDING_MODEL']}"
finetuned_path = "model_training/model/"

#### Raw model w/o prompting
Let's 1st check the stats for the raw model without any pormpting technique

In [16]:
raw_model = FlagModel(
    raw_name, 
    query_instruction_for_retrieval="",
    devices=[0],
    use_fp16=True
)


results, _ = Validator.search(raw_model, queries_text, corpus_text, queries)
eval_res = evaluate_metrics(qrels_dict, results, k_values)
mrr = evaluate_mrr(qrels_dict, results, k_values)

for res in eval_res:
    print(res)
print(mrr)

pre tokenize: 100%|██████████| 12/12 [00:00<00:00, 31.16it/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 94/94 [00:01<00:00, 91.57it/s]


defaultdict(<class 'list'>, {'NDCG@1': 0.62016, 'NDCG@3': 0.6928, 'NDCG@5': 0.71499, 'NDCG@10': 0.73232})
defaultdict(<class 'list'>, {'MAP@1': 0.55161, 'MAP@3': 0.64691, 'MAP@5': 0.664, 'MAP@10': 0.67436})
defaultdict(<class 'list'>, {'Recall@1': 0.55161, 'Recall@3': 0.74314, 'Recall@5': 0.80182, 'Recall@10': 0.85531})
defaultdict(<class 'list'>, {'P@1': 0.62016, 'P@3': 0.29194, 'P@5': 0.19339, 'P@10': 0.10621})
defaultdict(<class 'list'>, {'MRR@1': 0.62016, 'MRR@3': 0.70572, 'MRR@5': 0.71795, 'MRR@10': 0.72384})


We can see that the metrics @1 and @3 are standard for a model that hasn't been finetuned in out task nor is being used correctly as it's not preceeded by a prompt

### Raw model with prompting

In [17]:
prompted_raw_model = FlagModel(
    raw_name, 
    query_instruction_for_retrieval=CONFIG['QUERY_INSTRUCTION_AT_RETRIEVAL'],
    devices=[0],
    use_fp16=True
)


results, _ = Validator.search(prompted_raw_model, queries_text, corpus_text, queries)
eval_res = evaluate_metrics(qrels_dict, results, k_values)
mrr = evaluate_mrr(qrels_dict, results, k_values)

for res in eval_res:
    print(res)
print(mrr)

pre tokenize: 100%|██████████| 12/12 [00:00<00:00, 109.04it/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 94/94 [00:01<00:00, 92.55it/s]


defaultdict(<class 'list'>, {'NDCG@1': 0.63718, 'NDCG@3': 0.705, 'NDCG@5': 0.72482, 'NDCG@10': 0.74046})
defaultdict(<class 'list'>, {'MAP@1': 0.56459, 'MAP@3': 0.65901, 'MAP@5': 0.67493, 'MAP@10': 0.68473})
defaultdict(<class 'list'>, {'Recall@1': 0.56459, 'Recall@3': 0.75307, 'Recall@5': 0.80703, 'Recall@10': 0.85567})
defaultdict(<class 'list'>, {'P@1': 0.63718, 'P@3': 0.29717, 'P@5': 0.19546, 'P@10': 0.10684})
defaultdict(<class 'list'>, {'MRR@1': 0.63718, 'MRR@3': 0.7189, 'MRR@5': 0.73017, 'MRR@10': 0.7352})


When we add a prompt we see increases in the metrics, mostly @3, but let's check now our fine tuned model

### Finetuned model with prompting

In [18]:
ft_model = FlagModel(
    finetuned_path, 
    query_instruction_for_retrieval=CONFIG['QUERY_INSTRUCTION_AT_RETRIEVAL'],
    devices=[0],
    use_fp16=True
)

results, corpus_embeddings = Validator.search(ft_model, queries_text, corpus_text, queries)
eval_res = evaluate_metrics(qrels_dict, results, k_values)
mrr = evaluate_mrr(qrels_dict, results, k_values)

for res in eval_res:
    print(res)
print(mrr)

pre tokenize: 100%|██████████| 12/12 [00:00<00:00, 28.85it/s] 
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 94/94 [00:01<00:00, 92.15it/s]


defaultdict(<class 'list'>, {'NDCG@1': 0.78605, 'NDCG@3': 0.82023, 'NDCG@5': 0.82805, 'NDCG@10': 0.83965})
defaultdict(<class 'list'>, {'MAP@1': 0.70468, 'MAP@3': 0.7826, 'MAP@5': 0.79265, 'MAP@10': 0.80183})
defaultdict(<class 'list'>, {'Recall@1': 0.70468, 'Recall@3': 0.84114, 'Recall@5': 0.86802, 'Recall@10': 0.90394})
defaultdict(<class 'list'>, {'P@1': 0.78605, 'P@3': 0.33311, 'P@5': 0.21155, 'P@10': 0.11452})
defaultdict(<class 'list'>, {'MRR@1': 0.78605, 'MRR@3': 0.84001, 'MRR@5': 0.84423, 'MRR@10': 0.84715})


We see that the metrics increased greatly (eg: NDCG@1 increased +20% and NDCG@3 +17%). We can say that the embedding fine tune worked correclty and we ended up with a decent model to be used in a RAG system.

---

# Ingest into OpenSearch

Finally we will ingest our finetuned embeddings into opensearch so the can be used at inference time

In [22]:
from model_training.utils import OpenSearchManager
index_name = 'embedding-finetuned-v1'
host = CONFIG['OPENSEARCH_INDEX_URL'].removeprefix('https://').removesuffix('/'+index_name)
opens_mngr = OpenSearchManager(host)

In [23]:
opens_mngr.create_index(index_name)

In [24]:
opens_mngr.bulk_ingestion(index_name, corpus_text, corpus_embeddings)

Now we are ready to consume these vectors at inference time